In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'P100SXM2'
file_name = 'P100SXM2'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/2/benchmark_dense__20181119.pkl' %file_name))

for i in range(3,3):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20181119.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.784080,2037.468080,2042.244520,0.0,32.0,0.732888,0.701663,0.679047,0.019520,1.365308e+08,...,0.083280,0.083080,0.084800,0.493120,0.085440,0.085040,0.251960,0.249560,0.251560,0.246920
std,18.460976,1187.081709,1183.780913,0.0,0.0,0.395866,0.391203,0.392509,0.009891,1.597323e+08,...,0.276311,0.276009,0.278589,0.499963,0.279541,0.278947,0.434147,0.432767,0.433918,0.431228
min,1.000000,1.000000,1.000000,0.0,32.0,0.346479,0.326743,0.274577,0.000976,7.600000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,996.000000,1011.000000,0.0,32.0,0.456280,0.428758,0.402660,0.012755,2.204244e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,2043.000000,2035.000000,0.0,32.0,0.592036,0.558889,0.538108,0.017729,7.419462e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3065.250000,3077.250000,0.0,32.0,0.845661,0.812736,0.792259,0.024069,1.949631e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.000000,0.0,32.0,3.094482,3.024440,2.880960,0.117104,1.031366e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [4]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std
0,33,7,4,757,1125,4,1,32,1,True,6,0,12.488003,12.476168,12.495551,0.006848
1,63,480,2,2,14,4,0,32,2,True,0,0,2.283158,2.264562,2.293782,0.009580
2,57,180,7,28,13,3,1,32,1,False,0,0,4.139543,4.116368,4.147892,0.010720
3,55,443,7,14,9,3,0,32,2,True,5,0,74.082556,74.069028,74.109120,0.013753
4,5,27,7,181,74,1,0,32,2,True,0,0,0.822964,0.733547,0.827913,0.036228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,28,393,3,24,3,1,1,32,1,False,0,0,9.403720,9.394207,9.433041,0.013767
1496,20,479,7,11,2,3,0,32,3,True,6,0,25.297766,25.288801,25.320382,0.010814
1497,46,213,6,2,42,2,0,32,2,True,0,0,2.253609,2.249255,2.271810,0.008707
1498,21,450,3,8,16,3,0,32,3,False,4,0,11.259799,11.239190,11.284690,0.015426


In [5]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [6]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [7]:
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
0,33,7,4,757,1125,4,1,32,1,0,...,0,0,0,0,1,0,0,1,0,0
1,63,480,2,2,14,4,0,32,1,0,...,0,0,0,1,0,0,0,0,0,1
2,57,180,7,28,13,3,1,32,0,0,...,0,0,0,1,0,0,0,1,0,0
3,55,443,7,14,9,3,0,32,1,0,...,0,1,0,0,0,0,0,0,0,1
4,5,27,7,181,74,1,0,32,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,28,393,3,24,3,1,1,32,0,0,...,0,0,0,1,0,0,0,1,0,0
1496,20,479,7,11,2,3,0,32,1,0,...,0,0,0,0,1,0,0,0,1,0
1497,46,213,6,2,42,2,0,32,1,0,...,0,0,0,1,0,0,0,0,0,1
1498,21,450,3,8,16,3,0,32,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,5000.000000,5000.000000,5000.00000,5000.000000,5000.00000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,32.340200,257.037400,4.01820,63.593600,62.87780,2.510200,0.503600,32.0,0.503400,0.250000,...,0.083200,0.078200,0.076000,0.493400,0.084400,0.088800,0.249600,0.253400,0.249600,0.247400
std,18.683315,148.262388,1.98994,311.070565,296.26697,1.122563,0.500037,0.0,0.500038,0.433056,...,0.276212,0.268513,0.265025,0.500006,0.278015,0.284483,0.432825,0.435001,0.432825,0.431544
min,1.000000,1.000000,1.00000,1.000000,1.00000,1.000000,0.000000,32.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,130.000000,2.00000,10.000000,10.00000,2.000000,0.000000,32.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32.000000,256.000000,4.00000,19.000000,19.00000,3.000000,1.000000,32.0,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,386.000000,6.00000,38.000000,38.00000,4.000000,1.000000,32.0,1.000000,0.250000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,64.000000,512.000000,7.00000,9321.000000,8290.00000,4.000000,1.000000,32.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))